# Getting protein name and other synonyms 

## Source: uniprot

In [3]:
#import library
import pandas as pd
import re
import csv
import json

In [ ]:
#install the library if you don't have it
!pip install bioservices
from bioservices import UniProt

In [3]:
#read csv file into dataframe
kinase_df = pd.read_csv('clean_human_kinase.csv', index_col=0)
kinase_df.tail()

,gene_name,uniprot_identifier,uniprot_number
family,,,
Atypical: PI3/PI4-kinase family,PRKDC,PRKDC_HUMAN,P78527
Atypical: PI3/PI4-kinase family,SMG1,SMG1_HUMAN,Q96Q15
Atypical: RIO-type Ser/Thr kinase family,RIOK1,RIOK1_HUMAN,Q9BRS2
Atypical: RIO-type Ser/Thr kinase family,RIOK2,RIOK2_HUMAN,Q9BVS4
Atypical: RIO-type Ser/Thr kinase family,RIOK3,RIOK3_HUMAN,O14730


In [7]:
#unprocessed uniprot_number
unprocessed_list = kinase_df['uniprot_number'].tolist()
unprocessed_list[:10]

['P31749',
 'P31751',
 'Q9Y243',
 'Q5VT25',
 'Q9Y5S2',
 'Q6DT37',
 'O14578',
 'Q09013',
 'Q15835',
 'P25098']

In [8]:
#turn gene name into a list
up_num_list = kinase_df['uniprot_number'].str.rstrip().tolist() #rstrip to remove the white spaces
up_num_list

['P31749',
 'P31751',
 'Q9Y243',
 'Q5VT25',
 'Q9Y5S2',
 'Q6DT37',
 'O14578',
 'Q09013',
 'Q15835',
 'P25098',
 'P35626',
 'P32298',
 'P34947',
 'P43250',
 'Q8WTQ7',
 'O95835',
 'Q9NRM7',
 'Q9Y2H9',
 'Q6P0Q8',
 'O60307',
 'O15021',
 'Q96GX5',
 'O15530',
 'Q6A1A2',
 'Q16512',
 'Q16513',
 'Q6P5Z2',
 'P17612',
 'P22694',
 'P22612',
 'P17252',
 'P05771',
 'Q05655',
 'Q02156',
 'P05129',
 'P24723',
 'P41743',
 'Q04759',
 'Q05513',
 'Q13976',
 'Q13237',
 'P51817',
 'O43930',
 'Q13464',
 'O75116',
 'Q15418',
 'Q15349',
 'P51812',
 'O75676',
 'O75582',
 'Q9UK32',
 'P23443',
 'Q9UBS0',
 'O00141',
 'Q9HBY8',
 'Q96BR1',
 'Q15208',
 'Q9Y2H1',
 'Q8TDC3',
 'Q8IWQ3',
 'Q14012',
 'Q8IU85',
 'Q96NX5',
 'Q9UQM7',
 'Q13554',
 'Q13557',
 'Q13555',
 'Q16566',
 'Q8NCB2',
 'O14936',
 'O14757',
 'O96017',
 'P53355',
 'Q9UIK4',
 'O43293',
 'O15075',
 'Q8N568',
 'Q9C098',
 'P57058',
 'O60229',
 'P49137',
 'Q16644',
 'Q8IW41',
 'Q9P0L2',
 'Q7KZI7',
 'P27448',
 'Q96L34',
 'Q14680',
 'Q9BUB5',
 'Q9HBH9',
 'Q15746',

In [13]:
#using bioservices
u = UniProt(verbose=False)
u.search("id:P31751", limit=1, columns="id,protein names,entry name")

'Entry\tProtein names\tEntry name\nP31751\tRAC-beta serine/threonine-protein kinase (EC 2.7.11.1) (Protein kinase Akt-2) (Protein kinase B beta) (PKB beta) (RAC-PK-beta)\tAKT2_HUMAN\n'

In [18]:
#split the output by multiple seps
example = u.search("id:Q5VT25", limit=1, columns="id,protein names, genes(PREFERRED), genes, entry name")
re.split('\t|\n', example)

['Entry',
 'Protein names',
 'Gene names  (primary )',
 'Gene names',
 'Entry name',
 'Q5VT25',
 'Serine/threonine-protein kinase MRCK alpha (EC 2.7.11.1) (CDC42-binding protein kinase alpha) (DMPK-like alpha) (Myotonic dystrophy kinase-related CDC42-binding kinase alpha) (MRCK alpha) (Myotonic dystrophy protein kinase-like alpha)',
 'CDC42BPA',
 'CDC42BPA KIAA0451',
 'MRCKA_HUMAN',
 '']

In [19]:
#retrieve the protein names
ex_str = re.split('\t|\n', example)
ex_str[6].split('(')[0].rstrip()

'Serine/threonine-protein kinase MRCK alpha'

In [20]:
#create a function to get the uniprot_number, gene_name, protein_name, gene_aliases
def get_meta(uniprot_num):
    """
    Takes in a uniprot number.
    Returns the uniprot_number, gene_name, protein_name, gene_aliases as a tuple.
    """
    query = "id:{}".format(uniprot_num)
    output = u.search(query, limit=1, columns="id,protein names, genes(PREFERRED), genes, entry name")
    tmp = re.split('\t|\n', output)
    protein_name = tmp[6].split('(')[0].rstrip()
    gene_name = tmp[7]
    gene_aliases = tmp[8].split()
    uniprot_entry = tmp[9]
    return(uniprot_num, gene_name, protein_name, gene_aliases, uniprot_entry)
get_meta('Q5VT25')

('Q5VT25',
 'CDC42BPA',
 'Serine/threonine-protein kinase MRCK alpha',
 ['CDC42BPA', 'KIAA0451'],
 'MRCKA_HUMAN')

In [21]:
#get all the protein names
total = []
unavailable = []
for num in up_num_list:
    tmp = get_meta(num)
    if len(tmp) < 5:
        unavailable.append(num)
    else:
        total.append(tmp)
print(len(unavailable))
print(total[:5])

0
[('P31749', 'AKT1', 'RAC-alpha serine/threonine-protein kinase', ['AKT1', 'PKB', 'RAC'], 'AKT1_HUMAN'), ('P31751', 'AKT2', 'RAC-beta serine/threonine-protein kinase', ['AKT2'], 'AKT2_HUMAN'), ('Q9Y243', 'AKT3', 'RAC-gamma serine/threonine-protein kinase', ['AKT3', 'PKBG'], 'AKT3_HUMAN'), ('Q5VT25', 'CDC42BPA', 'Serine/threonine-protein kinase MRCK alpha', ['CDC42BPA', 'KIAA0451'], 'MRCKA_HUMAN'), ('Q9Y5S2', 'CDC42BPB', 'Serine/threonine-protein kinase MRCK beta', ['CDC42BPB', 'KIAA1124'], 'MRCKB_HUMAN')]


In [22]:
#convert the list into dataframe
meta_df = pd.DataFrame(total, columns=['Uniprot_number','Gene_name','Protein_name','Gene_aliases', 'Uniprot_entry'])
meta_df.head()

,Uniprot_number,Gene_name,Protein_name,Gene_aliases,Uniprot_entry
0,P31749,AKT1,RAC-alpha serine/threonine-protein kinase,"[AKT1, PKB, RAC]",AKT1_HUMAN
1,P31751,AKT2,RAC-beta serine/threonine-protein kinase,[AKT2],AKT2_HUMAN
2,Q9Y243,AKT3,RAC-gamma serine/threonine-protein kinase,"[AKT3, PKBG]",AKT3_HUMAN
3,Q5VT25,CDC42BPA,Serine/threonine-protein kinase MRCK alpha,"[CDC42BPA, KIAA0451]",MRCKA_HUMAN
4,Q9Y5S2,CDC42BPB,Serine/threonine-protein kinase MRCK beta,"[CDC42BPB, KIAA1124]",MRCKB_HUMAN


In [23]:
#turn the meta df into csv file
meta_df.to_csv('meta_names.csv', header=True, index=False)

The metadata is now available

# Checking if Mo's kinase is within my list of human kinases

In [81]:
#read in mo's csv
kinase_substrate_df = pd.read_csv('clean_human_kinase_substrates.csv')
kinase_substrate_df.head()

,Unnamed: 0,GENE,KINASE,KIN_ACC_ID,SUBSTRATE,SUB_GENE_ID,SUB_ACC_ID,SUB_GENE,SUB_MOD_RSD,SITE_GRP_ID,SITE_+/-7_AA,DOMAIN
0,7,EIF2AK1,HRI,Q9BQI3,eIF2-alpha,1965.0,P05198,EIF2S1,S52,447635,MILLsELsRRRIRsI,S1
1,8,EIF2AK1,HRI,Q9BQI3,eIF2-alpha,1965.0,P05198,EIF2S1,S49,450210,IEGMILLsELsRRRI,S1
2,11,PRKCD,PKCD,Q05655,HDAC5,10014.0,Q9UQL6,HDAC5,S259,447995,FPLRkTAsEPNLKVR,NaN
3,12,PRKCD,PKCD,Q05655,PTPRA iso2,5786.0,P18433-2,PTPRA,S204,447612,PLLARSPsTNRKYPP,NaN
4,13,PRKCD,PKCD,Q05655,hnRNP K,3190.0,P61978,HNRNPK,S302,457408,GrGGrGGsrArNLPL,NaN


In [25]:
meta_df.head()

,Uniprot_number,Gene_name,Protein_name,Gene_aliases,Uniprot_entry
0,P31749,AKT1,RAC-alpha serine/threonine-protein kinase,"[AKT1, PKB, RAC]",AKT1_HUMAN
1,P31751,AKT2,RAC-beta serine/threonine-protein kinase,[AKT2],AKT2_HUMAN
2,Q9Y243,AKT3,RAC-gamma serine/threonine-protein kinase,"[AKT3, PKBG]",AKT3_HUMAN
3,Q5VT25,CDC42BPA,Serine/threonine-protein kinase MRCK alpha,"[CDC42BPA, KIAA0451]",MRCKA_HUMAN
4,Q9Y5S2,CDC42BPB,Serine/threonine-protein kinase MRCK beta,"[CDC42BPB, KIAA1124]",MRCKB_HUMAN


In [49]:
#turn meta_df into dictionary with gene_aliases as key and the gene_name as value
kinase_dict = {}
for index, row in meta_df.iterrows():
    kinase_dict[tuple(row['Gene_aliases'])] = row['Gene_name']
#     print(tuple(row['Gene_aliases']))
kinase_dict

{('AKT1', 'PKB', 'RAC'): 'AKT1',
 ('AKT2',): 'AKT2',
 ('AKT3', 'PKBG'): 'AKT3',
 ('CDC42BPA', 'KIAA0451'): 'CDC42BPA',
 ('CDC42BPB', 'KIAA1124'): 'CDC42BPB',
 ('CDC42BPG', 'DMPK2'): 'CDC42BPG',
 ('CIT', 'CRIK', 'KIAA0949', 'STK21'): 'CIT',
 ('DMPK', 'DM1PK', 'MDPK'): 'DMPK',
 ('GRK1', 'RHOK'): 'GRK1',
 ('GRK2', 'ADRBK1', 'BARK', 'BARK1'): 'GRK2',
 ('GRK3', 'ADRBK2', 'BARK2'): 'GRK3',
 ('GRK4', 'GPRK2L', 'GPRK4'): 'GRK4',
 ('GRK5', 'GPRK5'): 'GRK5',
 ('GRK6', 'GPRK6'): 'GRK6',
 ('GRK7', 'GPRK7'): 'GRK7',
 ('LATS1', 'WARTS'): 'LATS1',
 ('LATS2', 'KPM'): 'LATS2',
 ('MAST1', 'KIAA0973', 'SAST'): 'MAST1',
 ('MAST2', 'KIAA0807', 'MAST205'): 'MAST2',
 ('MAST3', 'KIAA0561'): 'MAST3',
 ('MAST4', 'KIAA0303'): 'MAST4',
 ('MASTL', 'GW', 'GWL', 'THC2'): 'MASTL',
 ('PDPK1', 'PDK1'): 'PDPK1',
 ('PDPK2P', 'PDPK2'): 'PDPK2P',
 ('PKN1', 'PAK1', 'PKN', 'PRK1', 'PRKCL1'): 'PKN1',
 ('PKN2', 'PRK2', 'PRKCL2'): 'PKN2',
 ('PKN3', 'PKNBETA'): 'PKN3',
 ('PRKACA', 'PKACA'): 'PRKACA',
 ('PRKACB',): 'PRKACB',
 ('P

In [ ]:
#turn mo's kinase_gene into a list
mo_kinase_gene = kinase_substrate_df['GENE'].to_list()
mo_kinase_gene

In [67]:
#turns the aliases into a list
kinase_list = []
for aliases in kinase_dict:
    for gene in aliases:
        kinase_list.append(gene)
kinase_list

['AKT1',
 'PKB',
 'RAC',
 'AKT2',
 'AKT3',
 'PKBG',
 'CDC42BPA',
 'KIAA0451',
 'CDC42BPB',
 'KIAA1124',
 'CDC42BPG',
 'DMPK2',
 'CIT',
 'CRIK',
 'KIAA0949',
 'STK21',
 'DMPK',
 'DM1PK',
 'MDPK',
 'GRK1',
 'RHOK',
 'GRK2',
 'ADRBK1',
 'BARK',
 'BARK1',
 'GRK3',
 'ADRBK2',
 'BARK2',
 'GRK4',
 'GPRK2L',
 'GPRK4',
 'GRK5',
 'GPRK5',
 'GRK6',
 'GPRK6',
 'GRK7',
 'GPRK7',
 'LATS1',
 'WARTS',
 'LATS2',
 'KPM',
 'MAST1',
 'KIAA0973',
 'SAST',
 'MAST2',
 'KIAA0807',
 'MAST205',
 'MAST3',
 'KIAA0561',
 'MAST4',
 'KIAA0303',
 'MASTL',
 'GW',
 'GWL',
 'THC2',
 'PDPK1',
 'PDK1',
 'PDPK2P',
 'PDPK2',
 'PKN1',
 'PAK1',
 'PKN',
 'PRK1',
 'PRKCL1',
 'PKN2',
 'PRK2',
 'PRKCL2',
 'PKN3',
 'PKNBETA',
 'PRKACA',
 'PKACA',
 'PRKACB',
 'PRKACG',
 'PRKCA',
 'PKCA',
 'PRKACA',
 'PRKCB',
 'PKCB',
 'PRKCB1',
 'PRKCD',
 'PRKCE',
 'PKCE',
 'PRKCG',
 'PKCG',
 'PRKCH',
 'PKCL',
 'PRKCL',
 'PRKCI',
 'DXS1179E',
 'PRKCQ',
 'PRKCT',
 'PRKCZ',
 'PKC2',
 'PRKG1',
 'PRKG1B',
 'PRKGR1A',
 'PRKGR1B',
 'PRKG2',
 'PRKGR2',
 '

In [68]:
#loop through the list to find any mo's gene that might not be available in mine
not_available=[]
available = []
for gene_query in mo_kinase_gene:
    if gene_query in available:
        continue
    elif gene_query in not_available:
        continue
    else:
        if gene_query in kinase_list:
            available.append(gene_query)
        else:
            not_available.append(gene_query)
print(len(available))
print(not_available)

340
['GTF2F1', 'PRKAG2', 'BCR/ABL', 'NME1', 'HSPA5', 'BCR', 'PIKFYVE', 'PHKA1', 'BAZ1B', 'TAF1', 'TGM2', 'ENPP3', 'PGK1', 'NME2', 'COL4A3BP', 'NPM/ALK', 'PRKAB1', 'PKM', 'BLVRA', 'CSNK2B', 'BRD4', 'FAM20C']


In [72]:
#output the unavailable into csv
unavailable_df = pd.DataFrame(not_available, columns=['unavailable_kinase'])
unavailable_df.to_csv('unavailable_mo_kinase.csv', index=False)

We found that there are 22 kinases in Mo's which are not available in the original human kinase list that Han compiled. we took a look at the kinases. We found that a few of them are not real human kinases in Uniprot and others are only kinases in disease state. Therefore we decided to drop them from the phosphosites (clean_human_kinase_substrates.csv).

# Dropping the unavailable kinases from Mo's phosphosite data

In [73]:
#Mo's kinase_substrate df
kinase_substrate_df.head()

,Unnamed: 0,GENE,KINASE,KIN_ACC_ID,SUBSTRATE,SUB_GENE_ID,SUB_ACC_ID,SUB_GENE,SUB_MOD_RSD,SITE_GRP_ID,SITE_+/-7_AA,DOMAIN
0,7,EIF2AK1,HRI,Q9BQI3,eIF2-alpha,1965.0,P05198,EIF2S1,S52,447635,MILLsELsRRRIRsI,S1
1,8,EIF2AK1,HRI,Q9BQI3,eIF2-alpha,1965.0,P05198,EIF2S1,S49,450210,IEGMILLsELsRRRI,S1
2,11,PRKCD,PKCD,Q05655,HDAC5,10014.0,Q9UQL6,HDAC5,S259,447995,FPLRkTAsEPNLKVR,NaN
3,12,PRKCD,PKCD,Q05655,PTPRA iso2,5786.0,P18433-2,PTPRA,S204,447612,PLLARSPsTNRKYPP,NaN
4,13,PRKCD,PKCD,Q05655,hnRNP K,3190.0,P61978,HNRNPK,S302,457408,GrGGrGGsrArNLPL,NaN


In [82]:
#replace the GENE that are not in Han's kinase list
for i in range(len(kinase_substrate_df['GENE'])):
    gene = kinase_substrate_df.iloc[i]['GENE']
    if gene in not_available:
        kinase_substrate_df.loc[gene, 'GENE'] = 'NULL'
kinase_substrate_df

,Unnamed: 0,GENE,KINASE,KIN_ACC_ID,SUBSTRATE,SUB_GENE_ID,SUB_ACC_ID,SUB_GENE,SUB_MOD_RSD,SITE_GRP_ID,SITE_+/-7_AA,DOMAIN
0,7.0,EIF2AK1,HRI,Q9BQI3,eIF2-alpha,1965.0,P05198,EIF2S1,S52,447635.0,MILLsELsRRRIRsI,S1
1,8.0,EIF2AK1,HRI,Q9BQI3,eIF2-alpha,1965.0,P05198,EIF2S1,S49,450210.0,IEGMILLsELsRRRI,S1
2,11.0,PRKCD,PKCD,Q05655,HDAC5,10014.0,Q9UQL6,HDAC5,S259,447995.0,FPLRkTAsEPNLKVR,NaN
3,12.0,PRKCD,PKCD,Q05655,PTPRA iso2,5786.0,P18433-2,PTPRA,S204,447612.0,PLLARSPsTNRKYPP,NaN
4,13.0,PRKCD,PKCD,Q05655,hnRNP K,3190.0,P61978,HNRNPK,S302,457408.0,GrGGrGGsrArNLPL,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
PKM,NaN,NULL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BLVRA,NaN,NULL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CSNK2B,NaN,NULL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BRD4,NaN,NULL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [86]:
#dropping the rows with NULL in GENE
kinase_substrate_df = kinase_substrate_df[kinase_substrate_df.GENE != 'NULL']
kinase_substrate_df.reset_index(drop=True)

,Unnamed: 0,GENE,KINASE,KIN_ACC_ID,SUBSTRATE,SUB_GENE_ID,SUB_ACC_ID,SUB_GENE,SUB_MOD_RSD,SITE_GRP_ID,SITE_+/-7_AA,DOMAIN
0,7.0,EIF2AK1,HRI,Q9BQI3,eIF2-alpha,1965.0,P05198,EIF2S1,S52,447635.0,MILLsELsRRRIRsI,S1
1,8.0,EIF2AK1,HRI,Q9BQI3,eIF2-alpha,1965.0,P05198,EIF2S1,S49,450210.0,IEGMILLsELsRRRI,S1
2,11.0,PRKCD,PKCD,Q05655,HDAC5,10014.0,Q9UQL6,HDAC5,S259,447995.0,FPLRkTAsEPNLKVR,NaN
3,12.0,PRKCD,PKCD,Q05655,PTPRA iso2,5786.0,P18433-2,PTPRA,S204,447612.0,PLLARSPsTNRKYPP,NaN
4,13.0,PRKCD,PKCD,Q05655,hnRNP K,3190.0,P61978,HNRNPK,S302,457408.0,GrGGrGGsrArNLPL,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
10976,18451.0,NUAK1,NuaK1,O60285,MYPT1,4659.0,O14974,PPP1R12A,S910,1983104.0,sLLGRsGsysyLEER,NaN
10977,18452.0,NUAK1,NuaK1,O60285,LATS1,9113.0,O95835,LATS1,S464,451855.0,NIPVRsNsFNNPLGN,NaN
10978,18453.0,ULK2,ULK2,Q8IYT8,SEC16A,9919.0,O15027,SEC16A,S846,55578720.0,LAQPINFsVSLSNSH,NaN
10979,18454.0,ULK2,ULK2,Q8IYT8,DENND3,22898.0,A2RUS2,DENND3,S472,7997148.0,THRRMVVsMPNLQDI,NaN


In [87]:
kinase_substrate_df.head()

,Unnamed: 0,GENE,KINASE,KIN_ACC_ID,SUBSTRATE,SUB_GENE_ID,SUB_ACC_ID,SUB_GENE,SUB_MOD_RSD,SITE_GRP_ID,SITE_+/-7_AA,DOMAIN
0,7.0,EIF2AK1,HRI,Q9BQI3,eIF2-alpha,1965.0,P05198,EIF2S1,S52,447635.0,MILLsELsRRRIRsI,S1
1,8.0,EIF2AK1,HRI,Q9BQI3,eIF2-alpha,1965.0,P05198,EIF2S1,S49,450210.0,IEGMILLsELsRRRI,S1
2,11.0,PRKCD,PKCD,Q05655,HDAC5,10014.0,Q9UQL6,HDAC5,S259,447995.0,FPLRkTAsEPNLKVR,NaN
3,12.0,PRKCD,PKCD,Q05655,PTPRA iso2,5786.0,P18433-2,PTPRA,S204,447612.0,PLLARSPsTNRKYPP,NaN
4,13.0,PRKCD,PKCD,Q05655,hnRNP K,3190.0,P61978,HNRNPK,S302,457408.0,GrGGrGGsrArNLPL,NaN


In [93]:
kinase_substrate_df=kinase_substrate_df.drop('Unnamed: 0',axis=1)
kinase_substrate_df

,GENE,KINASE,KIN_ACC_ID,SUBSTRATE,SUB_GENE_ID,SUB_ACC_ID,SUB_GENE,SUB_MOD_RSD,SITE_GRP_ID,SITE_+/-7_AA,DOMAIN
0,EIF2AK1,HRI,Q9BQI3,eIF2-alpha,1965.0,P05198,EIF2S1,S52,447635.0,MILLsELsRRRIRsI,S1
1,EIF2AK1,HRI,Q9BQI3,eIF2-alpha,1965.0,P05198,EIF2S1,S49,450210.0,IEGMILLsELsRRRI,S1
2,PRKCD,PKCD,Q05655,HDAC5,10014.0,Q9UQL6,HDAC5,S259,447995.0,FPLRkTAsEPNLKVR,NaN
3,PRKCD,PKCD,Q05655,PTPRA iso2,5786.0,P18433-2,PTPRA,S204,447612.0,PLLARSPsTNRKYPP,NaN
4,PRKCD,PKCD,Q05655,hnRNP K,3190.0,P61978,HNRNPK,S302,457408.0,GrGGrGGsrArNLPL,NaN
...,...,...,...,...,...,...,...,...,...,...,...
10976,NUAK1,NuaK1,O60285,MYPT1,4659.0,O14974,PPP1R12A,S910,1983104.0,sLLGRsGsysyLEER,NaN
10977,NUAK1,NuaK1,O60285,LATS1,9113.0,O95835,LATS1,S464,451855.0,NIPVRsNsFNNPLGN,NaN
10978,ULK2,ULK2,Q8IYT8,SEC16A,9919.0,O15027,SEC16A,S846,55578720.0,LAQPINFsVSLSNSH,NaN
10979,ULK2,ULK2,Q8IYT8,DENND3,22898.0,A2RUS2,DENND3,S472,7997148.0,THRRMVVsMPNLQDI,NaN


In [94]:
#output into a csv
kinase_substrate_df.to_csv('mohan_kinase_substrate.csv', index=False)

# Cleaning up the duplicates in the aliases and looking non-self-referencing aliases

## Look for non-self-referencing aliases

It is found that some entry has duplicates within the list of aliases for each kinase gene name. The following codes is to clean up the duplicates and to make sure that the alias list for each entry has its own kinase gene name for the purpose of query after building the database

In [2]:
#loading the csv into dataframe
meta_names_df = pd.read_csv("meta_names.csv")
meta_names_df.head()

,Uniprot_number,Gene_name,Protein_name,Gene_aliases,Uniprot_entry
0,P31749,AKT1,RAC-alpha serine/threonine-protein kinase,"['AKT1', 'PKB', 'RAC']",AKT1_HUMAN
1,P31751,AKT2,RAC-beta serine/threonine-protein kinase,['AKT2'],AKT2_HUMAN
2,Q9Y243,AKT3,RAC-gamma serine/threonine-protein kinase,"['AKT3', 'PKBG']",AKT3_HUMAN
3,Q5VT25,CDC42BPA,Serine/threonine-protein kinase MRCK alpha,"['CDC42BPA', 'KIAA0451']",MRCKA_HUMAN
4,Q9Y5S2,CDC42BPB,Serine/threonine-protein kinase MRCK beta,"['CDC42BPB', 'KIAA1124']",MRCKB_HUMAN


In [8]:
#to find out if there's any gene_aliases that lack of self-referencing name
#import csv 
#import json
no_self_reference = []
with open('meta_names.csv') as f: #open the file
    reader = csv.DictReader(f) #use DictReader to read into dictionary
    for row in reader:
        gene_name = row["Gene_name"] 
        gene_aliases = json.loads(row["Gene_aliases"].replace("'", '"')) #read the gene_aliases using json because DictReader returns it as a string of list of strings
        print('{}: {}'.format(gene_name, gene_aliases))
        if gene_name in gene_aliases:
            continue
        else:
            no_self_reference.append(gene_name)
print(no_self_reference)

AKT1: ['AKT1', 'PKB', 'RAC']
AKT2: ['AKT2']
AKT3: ['AKT3', 'PKBG']
CDC42BPA: ['CDC42BPA', 'KIAA0451']
CDC42BPB: ['CDC42BPB', 'KIAA1124']
CDC42BPG: ['CDC42BPG', 'DMPK2']
CIT: ['CIT', 'CRIK', 'KIAA0949', 'STK21']
DMPK: ['DMPK', 'DM1PK', 'MDPK']
GRK1: ['GRK1', 'RHOK']
GRK2: ['GRK2', 'ADRBK1', 'BARK', 'BARK1']
GRK3: ['GRK3', 'ADRBK2', 'BARK2']
GRK4: ['GRK4', 'GPRK2L', 'GPRK4']
GRK5: ['GRK5', 'GPRK5']
GRK6: ['GRK6', 'GPRK6']
GRK7: ['GRK7', 'GPRK7']
LATS1: ['LATS1', 'WARTS']
LATS2: ['LATS2', 'KPM']
MAST1: ['MAST1', 'KIAA0973', 'SAST']
MAST2: ['MAST2', 'KIAA0807', 'MAST205']
MAST3: ['MAST3', 'KIAA0561']
MAST4: ['MAST4', 'KIAA0303']
MASTL: ['MASTL', 'GW', 'GWL', 'THC2']
PDPK1: ['PDPK1', 'PDK1']
PDPK2P: ['PDPK2P', 'PDPK2']
PKN1: ['PKN1', 'PAK1', 'PKN', 'PRK1', 'PRKCL1']
PKN2: ['PKN2', 'PRK2', 'PRKCL2']
PKN3: ['PKN3', 'PKNBETA']
PRKACA: ['PRKACA', 'PKACA']
PRKACB: ['PRKACB']
PRKACG: ['PRKACG']
PRKCA: ['PRKCA', 'PKCA', 'PRKACA']
PRKCB: ['PRKCB', 'PKCB', 'PRKCB1']
PRKCD: ['PRKCD']
PRKCE: ['PRKCE',

The codes above showed that each gene has a self-referencing gene name in the list of aliases.

## Look for duplicates

In [9]:
gene_alias_list = []
duplicates = []
with open('meta_names.csv') as f: #open the file
    reader = csv.DictReader(f) #use DictReader to read into dictionary
    for row in reader:
        gene_aliases = json.loads(row["Gene_aliases"].replace("'", '"'))
        for alias in gene_aliases:
            if alias in gene_alias_list:
                duplicates.append(alias)
            else:
                gene_alias_list.append(alias)
duplicates

['PRKACA',
 'CAMK',
 'GPRK7',
 'MLCK',
 'CDK11',
 'NEK8',
 'NIK',
 'PAK1',
 'PAK5',
 'STK2',
 'STK3',
 'CAK',
 'TYRO4',
 'STK1',
 'STK1',
 'MAK',
 'PDK1']

The list above shows that there are a number of gene aliases duplicated. The duplicates will be removed from the latter entries.